# ADA by ADM by Student Report

In [1]:
start_date = str(input('Input begin date and end date in YYYY-MM-DD format ')).strip()
end_date = str(input('Input end date in YYYY-MM-DD format ')).strip()
month_num = str(input('Input month number '))


# 12/4/2023-1/19/2024

#Example 
#start_date = '2023-08-14'
#end_date = '2023-09-08'
#month_num = '1'

# Month 2 is from 9/11/2023-10/06/2023
# Month 3 is from 10/9/2023-11/3/2023
# Month 4 range is from 11/6/2023 - 12/1/2023

#Final files can be found at 
# P:\Knowledge Management\Attendance & Membership\Authorizer Reports\LAUSD Reports\2023-24\Backup_Reports\ADA_ADM_Date_and_Attendance_Summary_Reports

In [3]:
import acme_powerschool
import pandas as pd
import json
import logging
import os

server = "ps.greendot.org"
client_id="9c8cf4eb-04e9-4903-870d-ac1d1e1fc807"
client_secret="9824eaab-7cca-4534-92d5-3f93358609f7"

# ----------------------------------------------------------------------------------Set logging options------------------

# set logging options - level INFO for normal operation and DEBUG for troubleshooting
logging.basicConfig(
    level=logging.INFO,
    format="[%(asctime)s] [%(levelname)8s] %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
    filename= os.getcwd() + "\\LAUSD_ADA_ADM_Student.log"
    )

# Create new section for log
logging.info(
    " \n"
    "-------------------------------------------------\n"
    f"Beginning query requests from {server}\n"
    "-------------------------------------------------"
)

# ------------------------------------------Instantiate instance of PS_endpoints class to make calls-----------------------

# want this to flow effortlessly in case it ever needed to be scaled from TN. 
class PS:
    
    query="org.greendot.attendance.adaadmbystudent.query"
    
    def __init__(self, client_secret, client_id):
        self.client_secret = client_secret
        self.client_id = client_id
        self.ps = acme_powerschool.api(server,client_id=self.client_id, client_secret=self.client_secret)
        
    
    def getPSData(self, date_begin, date_end):
        
        args = {'datebegin': date_begin, 'dateend': date_end}
    
        query_columns = {
        "student_number": "Student Number",
        "state_student_number": "State Student Number",
        "name": "Name",
        "grade": "Grade",
        "school_number": "School Number",
        "school": "School",
        "entry_date": "Entry Date",
        "exit_date": "Exit Date",
        "membership": "Membership",
        "attendance": "Attendance",
        "absence": "Absence",
        "off_track_days": "Off_Track_Days",
        "days_ne": "Days N/E",
        "districtofresidence": "DistrictOfResidence",
        "districtofresidencename": "DistrictOfResidenceName"
        }

        # Stream results by posting the date_begin & date_end to the created endpoint
        logging.info(f"Requesting query: {PS.query}")
        response = self.ps.post(f"/ws/schema/query/{PS.query}?pagesize=0", stream=True, data=json.dumps(args))

        # Error in streaming
        if response.status_code != 200:
            logging.error(f"PowerQuery failure: {response.status_code} {response.reason} {response.text}")

        # Streaming OK
        else:
            data = response.json()
            logging.info(f"Retrieved {len(data['record'])} records")

            # Read into pandas df
            df = pd.DataFrame.from_dict( data['record'] )

            # reorder columns
            df = df.reindex(columns=query_columns.keys())

            # rename columns
            df = df.rename(columns=query_columns)  
            #change data type for pivot table
            df['Attendance'] = df['Attendance'].astype(float)
            df.loc[:, 'DistrictOfResidenceName'] = df['DistrictOfResidenceName'].fillna('Blank')

            # Output data head
            logging.debug("Data preview:")
            logging.debug(df.head())

            return df

    
instance = PS(client_secret, client_id)
df = instance.getPSData(start_date, end_date)

#Due to multiple enrollments of starting and stopping, must drop duplicates
columns_to_check = ['Student Number', 'State Student Number', 'School Number']
df = df.drop_duplicates(subset=columns_to_check)

# ---------------------------------------pivot tables by district-----------------------------------------------------

def piv_by_district():
   
    path_send = r'P:\Knowledge Management\Attendance & Membership - TO DO - Vicky\Authorizer Reports\LAUSD Reports\2023-24\Backup_Reports\ADA_ADM_Student_Reports\ADA_ADM_Student_Reports_Month_{}'.format(month_num)
    
    if not os.path.exists(path_send):
        os.makedirs(path_send)
    
    schools = list(df['School'].unique())
    
    for i, name in enumerate(schools):
        
        sub = df.loc[df['School'] == schools[i]]
        
        piv = sub.pivot_table(values='Attendance', index='DistrictOfResidenceName', aggfunc='sum', margins = True, margins_name = 'Grand Total')
        piv['Attendance'] = piv['Attendance'].astype(int)

        piv.reset_index(inplace = True)
        piv.rename(columns = {'Attendance': 'Sum of Attendance'}, inplace = True)
        
        # create a Pandas Excel writer using XlsxWriter as the engine
        writer = pd.ExcelWriter(path_send + f'\\{name}_ADA_by_ADM_by_Student_District_Month_{month_num}.xlsx', engine='xlsxwriter')
        
        # write each DataFrame to a different sheet
        piv.to_excel(writer, sheet_name='Days of Attendance by District', index = False)
        sub.to_excel(writer, sheet_name='ADA by ADM by Student', index=False)
        
        logging.info(f'Sent {name}_ADA_by_ADM_by_Student_District_Month_{month_num}.xlsx')

        # save the Excel file
        writer.save()
        
        
piv_by_district()


C:\Users\SAMUEL~1.TAY\AppData\Local\Temp/ipykernel_30616/1260954845.py:134: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()
C:\Users\SAMUEL~1.TAY\AppData\Local\Temp/ipykernel_30616/1260954845.py:134: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()
C:\Users\SAMUEL~1.TAY\AppData\Local\Temp/ipykernel_30616/1260954845.py:134: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()
C:\Users\SAMUEL~1.TAY\AppData\Local\Temp/ipykernel_30616/1260954845.py:134: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()
C:\Users\SAMUEL~1.TAY\AppData\Local\Temp/ipykernel_30616/1260954845.py:134: FutureWarning: save is not part of the public API, usage